<a href="https://colab.research.google.com/github/ednavivianasegura/ERAP_Curso_R/blob/main/ERAP_R_Course_Modulo_IV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Título: Manipulación de datos con R - Módulo IV (Exploración de datos y Estadística descriptiva)

Autor(es): Edna Viviana Segura Alvarado - Hans Mauricio Carrillo Hernández

Fecha: 2025-06

Institución: Universidad de La Rioja

# Estadística Descriptiva

La estadística descriptiva se encarga de recolectar, organizar, representar y resumir la información contenida en un conjunto de datos. Su objetivo es **describir las características fundamentales** de una o más variables, sin llegar a realizar inferencias o generalizaciones.

---

## Clasificación de las variables

| Tipo de variable     | Subtipo              | Ejemplo                            |
|----------------------|----------------------|------------------------------------|
| **Cualitativa**      | Nominal              | Sexo (masculino/femenino)         |
|                      | Ordinal              | Nivel educativo (bajo/medio/alto) |
| **Cuantitativa**     | Discreta             | Número de hijos (0, 1, 2…)         |
|                      | Continua             | Edad, ingreso mensual, peso       |

---

### Enfoque de análisis:

| Enfoque           | Descripción |
|-------------------|-------------|
| **Univariado**    | Análisis de una sola variable. Se explora su distribución, tendencia central, dispersión y forma. |
| **Bivariado**     | Análisis conjunto de dos variables para identificar relaciones o asociaciones. |
| *(Multivariado)*  | Análisis simultáneo de tres o más variables (fuera del alcance de esta sección). |

---

## Estadística Descriptiva Univariada

1. Para Variables **Cualitativas**:

Se resumen usando tablas de frecuencia:

- **Frecuencia absoluta**: número de observaciones por categoría.
- **Frecuencia relativa**: proporción (o porcentaje) respecto al total.
- **Representaciones gráficas**: gráfico de barras, gráfico de torta.

2. Para Variables **Cuantitativas**:

Se utilizan:

- **Medidas de tendencia central**: media, mediana, moda.
- **Medidas de dispersión**: rango, varianza, desviación estándar, coeficiente de variación.
- **Medidas de forma**: asimetría (skewness), curtosis (kurtosis).
- **Visualizaciones**: histogramas, diagramas de caja (boxplots), Q-Q plots.

---

## Estadística Descriptiva Bivariada

1. Dos Variables cualitativas:
- Tablas de contingencia
- Gráficos de mosaico o de barras agrupadas
- Medidas de asociación (por ejemplo, prueba Chi²)

2. Una Variable Cualitativa y una Cuantitativa:
- Promedios o medianas por grupo
- Boxplots por categoría
- ANOVA exploratoria

3. Dos Variables Cuantitativas:
- Nube de puntos (scatterplot)
- Coeficiente de correlación (Pearson, Spearman)
- Covarianza
- Recta de regresión simple

---



In [ ]:
# @title Librerías
# Paquetes necesarios

# Función para instalar si no está instalado
instalar_si_no <- function(paquete) {
    if (!requireNamespace(paquete, quietly = TRUE)) {
    install.packages(paquete, repos = "https://cloud.r-project.org")
  }
  library(paquete, character.only = TRUE)
}

# Lista de paquetes a verificar
paquetes <- c("forcats","VIM","knitr","e1071","tidyr","dplyr","ggplot2","e1071", "visdat","reshape2", "janitor")


# Instalar y cargar todos
invisible(lapply(paquetes, instalar_si_no))

# Instalar librería empire
remotes::install_github("davidbiol/empire")



# Datos

Utilizaremos nuevamente el conjunto Sleep de la librería VIM trabajado durante el módulo II.



In [ ]:
# Cargar dataset 'sleep'
data("sleep")
sleep

## Análisis univariado:


### Medidas de Tendencia Central, posición y coeficiente de variación:

Utilizaremos la función summarise de dplyr para encontrar las medidas de tendencia central y de posición de la variable **BodyWgt**, que no tiene datos faltantes:



In [ ]:
# Solo seleccionamos variables numéricas y eliminamos NAs (aunque de antemanos sabemos que no tiene NAs)
sleep_limpio_BodyWgt <- sleep %>%
  select_if(is.numeric) %>%
  filter(!is.na(BodyWgt))


In [ ]:
# Calcular resumen estadístico de la variable BodyWgt
(tabla_resultados1<-sleep_limpio_BodyWgt %>%
  summarise(
    # Medidas de tendencia central
    media = mean(BodyWgt),          # Promedio
    mediana = median(BodyWgt),      # Mediana

    # Medidas de dispersión
    varianza = var(BodyWgt),        # Varianza
    desviacion = sd(BodyWgt),       # Desviación estándar
    minimo = min(BodyWgt),          # Valor mínimo
    maximo = max(BodyWgt),          # Valor máximo
    rango = max(BodyWgt) - min(BodyWgt),  # Rango total

    # Medidas de posición
    iqr = IQR(BodyWgt),             # Rango intercuartílico
    q1 = quantile(BodyWgt, 0.25),   # Primer cuartil (25%)
    q3 = quantile(BodyWgt, 0.75),   # Tercer cuartil (75%)
    p10 = quantile(BodyWgt, 0.10),  # Percentil 10
    p90 = quantile(BodyWgt, 0.90),  # Percentil 90
    p95 = quantile(BodyWgt, 0.95),  # Percentil 95

    # Cálculo de los límites para detección de outliers según regla de 1.5*IQR
    IQR = q3 - q1,                       # Vuelvo a calcular el IQR por claridad
    LI = q1 - 1.5 * IQR,                # Límite inferior para outliers
    LS = q3 + 1.5 * IQR,                # Límite superior para outliers

    # Conteo total y de valores atípicos
    total = n(),                        # Número total de observaciones
    outliers = sum(BodyWgt < LI | BodyWgt > LS),  # Total de valores atípicos
    proporcion_outliers = outliers / total,       # Proporción de outliers

    # Forma de la distribución
    asimetria = skewness(BodyWgt),     # Asimetría (skewness)
    curtosis = kurtosis(BodyWgt),       # Curtosis

    # Coeficiente de variación
    coef_variacion = (desviacion / media) * 100
  ) %>%

  # Transforma la tabla: convierte las columnas en filas
  pivot_longer(cols = everything(),
               names_to = "estadistico",   # Nombre del estadístico
               values_to = "valor") %>%    # Valor correspondiente

  # Redondea los valores a 2 decimales y elimina notación científica
  mutate(
    valor = format(round(valor, 2), scientific = FALSE)
  ))


In [ ]:
# Calcular moda con count y slice_max
sleep_limpio_BodyWgt %>%
  count(BodyWgt) %>% #cuenta los valores diferentes con respecto a BodyWgt
  arrange(desc(n)) %>%
  slice(1)  # Muestra la moda


¿Qué nos dicen estos valores?:

| Estadístico                     | Valor      | Interpretación                                                                                                                          |
| ------------------------------- | ---------- | --------------------------------------------------------------------------------------------------------------------------------------- |
| **media**                       | 198.79     | El peso promedio es muy alto, pero está distorsionado por valores extremos (como elefantes o ballenas).                                 |
| **mediana**                     | 3.34       | El valor central realista: la mitad de los animales pesa menos de 3.34 kg. Mucho menor que la media → **asimetría positiva**.           |
| **varianza**                    | 808485.13  | Muestra una altísima dispersión de los pesos. Valores grandes como 6654 lo elevan fuertemente.                                          |
| **desviación típica**           | 899.16     | Indica gran dispersión respecto a la media. Al igual que la varianza, está **muy afectada por los valores extremos**.                   |
| **mínimo**                      | 0.00       | Existen animales extremadamente ligeros (posiblemente error de redondeo o especie muy pequeña).                                         |
| **máximo**                      | 6654.00    | Hay especies con más de 6 toneladas de peso. Confirma la existencia de outliers muy altos.                                              |
| **rango**                       | 6654.00    | Es la diferencia entre el máximo y el mínimo. Muy alto → **alta variabilidad global**.                                                  |
| **IQR (rango intercuartílico)** | 47.60      | El 50% central de los datos está entre 0.6 y 48.2 kg. Mucho más representativo que el rango total.                                      |
| **Q1**                          | 0.60       | El 25% de los animales pesa menos de 0.6 kg.                                                                                            |
| **Q3**                          | 48.20      | El 75% pesa menos de 48.2 kg.                                                                                                           |
| **P10**                         | 0.08       | El 10% de los animales pesa menos de 80 gramos.                                                                                         |
| **P90**                         | 205.50     | El 90% pesa menos de 205.5 kg.                                                                                                          |
| **P95**                         | 518.20     | El 95% pesa menos de 518.2 kg.                                                                                                          |
| **LI (límite inferior)**        | -70.80     | No se usa aquí porque ningún peso puede ser negativo.                                                                                   |
| **LS (límite superior)**        | 119.61     | Todo valor mayor que este es considerado outlier según el criterio de 1.5 \* IQR.                                                       |
| **total**                       | 62.00      | Se analizaron 62 observaciones con datos completos para `BodyWgt`.                                                                      |
| **outliers**                    | 10.00      | Hay 10 valores considerados **atípicos**.                                                                                               |
| **proporción de outliers**      | 0.16 (16%) | Es **una proporción elevada**: casi 1 de cada 6 observaciones es un outlier.                                                            |
| **asimetría**                   | 6.25       | Muy por encima de 0 → **fuerte asimetría positiva**. Hay una larga cola hacia la derecha.                                               |
| **curtosis**                    | 40.60      | Muy superior a 3 → distribución **leptocúrtica**: concentrada alrededor de la mediana, con colas muy pesadas (muchos valores extremos). |
| **Coeficiente de variación**| 452.32%|La desviación típica es más de 4 veces la media.|


Validación gráfica:

In [ ]:
#Diagrama de barras de la variable BodyWht
ggplot(sleep_limpio_BodyWgt, aes(x = BodyWgt)) +
  geom_histogram(bins = 30, fill = "skyblue", color = "black") + #histograma con 30 bins
  labs(title = "Distribución del peso corporal del mamífero", x = "Peso (kg)", y = "Frecuencia")


In [ ]:
#Boxplot de la bariable BodyWgt
ggplot(sleep_limpio, aes(y = BodyWgt)) +
  geom_boxplot(fill = "lightgreen") +
  labs(title = "Boxplot del peso corporal", y = "Peso (kg)")

In [ ]:
#Boxplot sin valores extremos y modificando los límites del eje
ggplot(sleep_limpio, aes(y = BodyWgt)) +
  geom_boxplot(fill = "lightgreen", outlier.shape = NA) +
  labs(title = "Boxplot del peso corporal", y = "Peso (kg)")+
  ylim(0,7.5)

La variable **BodyWgt** tiene una distribución altamente asimétrica y con colas pesadas.

Las medidas robustas (mediana, IQR) son mucho más representativas que la media o la desviación típica.

Un 16% de los datos son outliers, todos por encima del límite superior.

Se recomienda para el análisis:
- Trabajar con logaritmos o escalas logarítmicas.
- Reportar mediana en lugar de media.

In [ ]:
# Filtrar valores mayores que 0 antes de aplicar el log
# Crear una nueva columna con logaritmo base 10
sleep_limpio_BodyWgt <- sleep_limpio %>%
  filter(BodyWgt > 0) %>%
  mutate(log_BodyWgt = log10(BodyWgt))


In [ ]:
# Calcular resumen estadístico de la variable BodyWgt
(tabla_resultados2 <- sleep_limpio_BodyWgt %>%
  summarise(
    # Medidas de tendencia central
    media = mean(log_BodyWgt),          # Promedio
    mediana = median(log_BodyWgt),      # Mediana

    # Medidas de dispersión
    varianza = var(log_BodyWgt),        # Varianza
    desviacion = sd(log_BodyWgt),       # Desviación estándar
    minimo = min(log_BodyWgt),          # Valor mínimo
    maximo = max(log_BodyWgt),          # Valor máximo
    rango = max(log_BodyWgt) - min(log_BodyWgt),  # Rango total

    # Medidas de posición
    iqr = IQR(log_BodyWgt),             # Rango intercuartílico
    q1 = quantile(log_BodyWgt, 0.25),   # Primer cuartil (25%)
    q3 = quantile(log_BodyWgt, 0.75),   # Tercer cuartil (75%)
    p10 = quantile(log_BodyWgt, 0.10),  # Percentil 10
    p90 = quantile(log_BodyWgt, 0.90),  # Percentil 90
    p95 = quantile(log_BodyWgt, 0.95),  # Percentil 95

    # Cálculo de los límites para detección de outliers según regla de 1.5*IQR
    IQR = q3 - q1,                       # Vuelvo a calcular el IQR por claridad
    LI = q1 - 1.5 * IQR,                # Límite inferior para outliers
    LS = q3 + 1.5 * IQR,                # Límite superior para outliers

    # Conteo total y de valores atípicos
    total = n(),                        # Número total de observaciones
    outliers = sum(log_BodyWgt < LI | log_BodyWgt > LS),  # Total de valores atípicos
    proporcion_outliers = outliers / total,       # Proporción de outliers

    # Forma de la distribución
    asimetria = skewness(log_BodyWgt),     # Asimetría (skewness)
    curtosis = kurtosis(log_BodyWgt),       # Curtosis

    # Coeficiente de variación
    coef_variacion = (desviacion / media) * 100
  ) %>%

  # Transforma la tabla: convierte las columnas en filas
  pivot_longer(cols = everything(),
               names_to = "estadistico",   # Nombre del estadístico
               values_to = "valor") %>%    # Valor correspondiente

  # Redondea los valores a 2 decimales y elimina notación científica
  mutate(
    valor = format(round(valor, 2), scientific = FALSE)
  ))


### Tablas para informes

Usamos de la librería knitr de R, la función kable que permite crear tablas en documentos R Markdown.

In [ ]:
# Mostrar tabla
kable(tabla_resultados1, caption = "\nResumen de Estadísticas Descriptivas,\nPosición y\nForma de la distribución:\nVariable original")
kable(tabla_resultados2, caption = "\nResumen de Estadísticas Descriptivas,\nPosición y\nForma de la distribución:\nVariable transformada")

In [ ]:
#histograma
ggplot(sleep_limpio_BodyWgt, aes(x = log_BodyWgt)) +
  geom_histogram(bins = 30, fill = "lightblue") +
  labs(title = "Distribución logarítmica del peso", x = "log10(BodyWgt)")
#boxplot
ggplot(sleep_limpio_BodyWgt, aes(y = log_BodyWgt)) +
  geom_boxplot(fill = "lightgreen") +
  labs(title = "Boxplot logarítmico del peso", y = "log10(BodyWgt)")


¿La variable **BodyWgt** sigue una distribución normal?

In [ ]:
# Crear gráfico: Histograma de la variable BodyWgt con curva normal superpuesta
ggplot(sleep_limpio_BodyWgt, aes(x = BodyWgt)) +  # Se define la base del gráfico con los datos y la variable BodyWgt en el eje x
  geom_histogram(
    aes(y = ..density..),     # Se cambia el eje y para que el histograma represente densidades y no frecuencias absolutas
    bins = 30,                # Número de barras del histograma
    fill = "lightblue",       # Color de relleno de las barras
    color = "black"           # Borde negro para las barras
  ) +
  stat_function(
    fun = dnorm,              # Se superpone una curva de densidad de distribución normal
    args = list(              # Parámetros de la curva normal
      mean = mean(sleep_limpio_BodyWgt$BodyWgt),  # Media muestral de BodyWgt
      sd = sd(sleep_limpio_BodyWgt$BodyWgt)       # Desviación estándar muestral de BodyWgt
    ),
    color = "red"             # Color rojo para la curva normal
  ) +
  labs(
    title = "¿Es normal la distribución de BodyWgt?"  # Título del gráfico
  )



In [ ]:
#Diagramas q-q plot:

qqnorm(sleep_limpio_BodyWgt$BodyWgt)
qqline(sleep_limpio_BodyWgt$BodyWgt, col = "red")


In [ ]:
#Test de normalidad
shapiro.test(sleep_limpio_BodyWgt$BodyWgt)

¿y la variable **log_BodyWgt**?

In [ ]:
# Crear gráfico: Histograma del logaritmo de BodyWgt con una curva normal superpuesta
ggplot(sleep_limpio_BodyWgt, aes(x = log_BodyWgt)) +  # Se establece la base del gráfico con la variable transformada log_BodyWgt en el eje x
  geom_histogram(
    aes(y = ..density..),     # Se ajusta el eje y para que muestre densidades (frecuencia relativa), necesario para comparar con la curva normal
    bins = 30,                # Número de intervalos del histograma
    fill = "lightblue",       # Color de las barras del histograma
    color = "black"           # Borde negro para distinguir las barras
  ) +
  stat_function(
    fun = dnorm,              # Función que dibuja una curva de densidad normal teórica
    args = list(
      mean = mean(sleep_limpio_BodyWgt$log_BodyWgt),  # Se especifica la media de los datos transformados
      sd = sd(sleep_limpio_BodyWgt$log_BodyWgt)       # Se especifica la desviación estándar de los datos transformados
    ),
    color = "red"             # Color rojo para la curva normal
  ) +
  labs(
    title = "¿Es normal la distribución de log_BodyWgt?"  # Título del gráfico, que plantea la hipótesis de normalidad
  )


In [ ]:
#qqplot varaible log
qqnorm(sleep_limpio_BodyWgt$log_BodyWgt)
qqline(sleep_limpio_BodyWgt$log_BodyWgt, col = "red")

In [ ]:
#test de normalidad
shapiro.test(sleep_limpio_BodyWgt$log_BodyWgt)

Podemos así ver la importancia del análisis inicial y de la conveniencia de usar transformaciones de las variables.

## Análisis de Correlación y Covarianza

Explorar relaciones entre variables cuantitativas, identificar patrones lineales y cuantificar la fuerza y dirección de esas relaciones mediante medidas como correlación y covarianza.

- **Correlación de Pearson (cor())**

Variante normalizada de la covarianza.

Valores que oscilan entre -1 y 1:

* +1: correlación perfecta positiva
* 0: sin relación lineal
* -1: correlación perfecta negativa

Supone linealidad y normalidad de las variables.

- **Correlación de Spearman**

Se basa en rangos → no asume normalidad ni linealidad.
Útil para datos ordinales o relaciones no lineales monótonas.

- **Covarianza (cov())**

Mide la tendencia conjunta de dos variables a aumentar o disminuir.

* Valor positivo: las variables tienden a crecer juntas.
* Valor negativo: una aumenta y la otra disminuye.

Muy sensible a la escala de las variables




### Correlación:

Comenzamos con el análisis de correlación

In [ ]:
# Comenzamos seleccionando solo variables numéricas completas
# recordemos que tenemos que convertir las trews variables pred, exp y danger en factores:

# Conversión de variables numéricas a factores (categorías)
sleep_limpio <- sleep %>%
  mutate(
    Pred = as.factor(Pred),
    Exp = as.factor(Exp),
    Danger = as.factor(Danger)
  )

(sleep_numerico <- sleep_limpio %>%
  select_if(is.numeric))

#Para no perder los datos, vamos a imputar con ridge, como hicimos en el módulo II:

# Aplicar imputación por regresión lineal penalizada (ridge)
(new_sleep_ridge <- empire::estimate_ridge(data = sleep_numerico, diff = 10, ridge_alpha = 0)$new_dat)



In [ ]:
# Correlación de Pearson
visdat::vis_cor(new_sleep_ridge, na_action = "complete.obs")

In [ ]:
# Correlación de Spearman
vis_cor(new_sleep_ridge, cor_method = "spearman", na_action = "complete.obs")

| Característica      | **Correlación de Pearson**            | **Correlación de Spearman**                                  |
| ------------------- | ------------------------------------- | ------------------------------------------------------------ |
| Tipo de relación    | Lineal                                | Monótona (creciente o decreciente, no necesariamente lineal) |
| Datos esperados     | Cuantitativos, normales, sin outliers | Cuantitativos u ordinales, **no requiere normalidad**        |
| Sensible a outliers |  Sí, mucho                           |  No, al trabajar con rangos                                 |
| Escala utilizada    | Escala original                       | Rangos (datos convertidos a posiciones)                      |


**Comparación visual entre ambas matrices**

* Matriz de Pearson:

Se observan correlaciones muy altas (casi 1) entre algunas variables como BodyWgt y BrainWgt.

También aparecen correlaciones negativas marcadas entre variables como Gest y Sleep.

Sin embargo, la intensidad puede estar amplificada por outliers o por relaciones no perfectamente lineales.

* Matriz de Spearman:

Las correlaciones siguen una tendencia similar, pero son más moderadas (es decir, no tan extremas).

Las relaciones como BrainWgt ~ BodyWgt siguen siendo fuertes, pero no perfectas.

Algunas correlaciones negativas están más suavizadas, lo cual puede reflejar que las relaciones no son lineales, pero sí monótonas.

                                                                                             
- **Lineal**:

Cuando los datos siguen aproximadamente una **línea recta**. Es decir, por cada incremento en X, Y cambia siempre en proporción constante.                                          

- **Monótona**:

Cuando una variable **siempre aumenta o disminuye con la otra**, pero no necesariamente en línea recta. Puede crecer más rápido, más lento.


**Podemos responder entonces:**

- ¿Hay colinealidad?
- ¿Qué variables parecen tener una relación no lineal?
- ¿Conviene usar Spearman si hay outliers?

----------------
- ¿Hay colinealidad?

Sí, se observa una posible colinealidad entre algunas variables, especialmente entre BodyWgt y BrainWgt, ya que su correlación de Pearson es muy alta (cercana a 1).
Esto indica que una variable puede predecir casi perfectamente a la otra en escala lineal, lo cual podría ser problemático en modelos de regresión múltiple, ya que colinealidad alta distorsiona los coeficientes y su interpretación.

Es recomendable en esos casos revisar el VIF (Variance Inflation Factor) si se va a usar en un modelo, o considerar eliminar o transformar una de las dos variables. **Se sale de los límites de este módulo**.

- ¿Qué variables parecen tener una relación no lineal?

Visualizando las relaciones con gráficos de dispersión, y comparando Pearson vs Spearman, se puede inferir que algunas relaciones —como BodyWgt con Sleep, o Gest con Span— pueden no seguir una relación estrictamente lineal.
Por ejemplo, Sleep parece reducirse con BodyWgt, pero no de manera constante, sino con mayor cambio en los pesos bajos.
Esto sugiere una relación monótona no lineal (por ejemplo, logarítmica o exponencial inversa), en la cual Pearson subestima o distorsiona la fuerza de la asociación.

- ¿Conviene usar Spearman si hay outliers?

Sí. La correlación de Spearman es más robusta frente a valores atípicos, ya que se basa en rangos y no en las diferencias absolutas entre los valores.
En el dataset sleep, se ha observado que la variable BodyWgt tiene alta asimetría y valores extremos, por lo que usar Spearman es más apropiado.
Además, si hay sospecha de relaciones no lineales pero monótonas, Spearman proporciona una estimación más fiel de la fuerza de asociación.

En resumen, Spearman es preferible cuando hay outliers o relaciones no lineales (lo dijimos arriba).

### Covarianza:

In [ ]:
# Paso 1: Convertimos la matriz de covarianza en data frame y la reestructuramos en formato largo

melted_cov <- as.data.frame(matriz_cov) %>%
  mutate(var1 = rownames(.)) %>%  # Creamos una columna con los nombres de fila
  pivot_longer(-var1,             # Convertimos las columnas restantes en pares clave-valor
               names_to = "var2",  # Nombre de la variable que representa las columnas originales
               values_to = "value")  # Valor de la covarianza entre var1 y var2


In [ ]:
# Paso 2: Visualizamos la matriz de covarianza como un mapa de calor con valores incluidos

ggplot(melted_cov, aes(x = var1, y = var2, fill = value)) +
  geom_tile(color = "white") +  # Crea las celdas del mapa de calor con borde blanco
  geom_text(aes(label = round(value, 2)), size = 3.5, color = "black") +  # Agrega los valores de covarianza redondeados sobre cada celda
  scale_fill_gradient2(low = "blue", mid = "white", high = "red", midpoint = 0) +  # Paleta de color para distinguir signo e intensidad
  labs(title = "Matriz de Covarianza",    # Título del gráfico
       x = "", y = "", fill = "Covarianza") +  # Etiquetas
  theme_minimal() +  # Tema limpio
  theme(
        legend.title = element_text(size = 12), #Modifica el tamaño del texto de la escala de la
        # covarianza, es decir, de la leyenda.
        axis.text.x = element_text(angle = 45, hjust = 1, size = 12),# Gira etiquetas del eje X para mejorar legibilidad
        axis.text.y = element_text(hjust = 1, size = 12) # modifica el tamaño del texto del eje y
)


¿Qué vemos?:

El valor numérico de la covarianza entre dos variables (p. ej. entre BodyWgt y BrainWgt) está asociado con un color rojo más intenso, indicando mayor covarianza positiva.

Los valores más cercanos a 0 (blanco o rosa muy claro) indican poca o nula relación lineal conjunta.


1. Covarianzas altas y positivas:

* BodyWgt con BrainWgt

* BodyWgt con Gest

* BrainWgt con Gest


Estas variables tienden a variar conjuntamente: cuando una aumenta, las otras también. Esto tiene sentido biológico, ya que animales más grandes (mayor BodyWgt) tienden a tener cerebros más grandes (BrainWgt) y gestaciones más largas (Gest).

2. Covarianzas negativas

* BodyWgt con Sleep
* BodyWgt con NonD
* Gest con Sleep


Estas relaciones indican que cuando una variable aumenta, la otra tiende a disminuir.

Por ejemplo:

Los animales más grandes tienden a dormir menos.
Los animales con mayor gestación también duermen menos (posiblemente especies más longevas, activas o expuestas).

3. Covarianzas cercanas a cero

* Dream con casi todas las demás variables.
* NonD con Dream, Gest, etc.


No hay una relación clara ni fuerte entre estas variables en términos de variación conjunta.

Es **importante* tener en cuenta que la covarianza depende de las unidades de las variables, por eso hay números muy grandes (como BrainWgt en gramos, por ejemplo).

Para comparar más fácilmente entre pares de variables, es mejor usar la correlación, que está normalizada entre -1 y 1.

Esta matriz sí es muy útil para detectar agrupaciones de variables altamente relacionadas, como BodyWgt, BrainWgt, Gest.

## Tablas de frecuencias y análisis de variables categóricas


En esta parte del módulo, nos enfocaremos en reconocer y analizar variables categóricas nominales y ordinales.

Para ello, elaboraremos tablas de frecuencias (absolutas y relativas).

Vamos a representar visualmente la distribución de variables categóricas.

Y exploraremos asociaciones entre variables categóricas (introducción a tablas de contingencia).

In [ ]:
sleep_limpio

In [ ]:
# Cargar el paquete janitor, que incluye funciones útiles para el manejo de tablas categóricas

# Tabla de frecuencias absolutas y relativas para la variable 'Danger'
sleep_limpio %>%                 # Objeto base: base de datos limpia
  tabyl(Danger) %>%              # `tabyl()` crea una tabla de frecuencias absolutas de la variable (función del paquete janitor)
  arrange(desc(n)) %>%         # Ordenar las filas por frecuencia absoluta (n) de mayor a menor
  adorn_pct_formatting()        # `adorn_pct_formatting()` convierte las proporciones en formato porcentaje y redondea (también de janitor)

# Tabla de frecuencias para la variable 'Pred'
sleep_limpio %>%
  tabyl(Pred) %>%                # Cuenta cuántas veces aparece cada categoría en 'Pred'
  arrange(desc(n)) %>%         # Ordenar las filas por frecuencia absoluta (n) de mayor a menor
  adorn_pct_formatting()        # Muestra también el porcentaje correspondiente a cada categoría

# Tabla de frecuencias para la variable 'Exp'
sleep_limpio %>%
  tabyl(Exp) %>%                 # Lo mismo, pero para la variable 'Exp'
  arrange(desc(n)) %>%         # Ordenar las filas por frecuencia absoluta (n) de mayor a menor
  adorn_pct_formatting()        # Devuelve una tabla con: categoría, frecuencia absoluta, porcentaje relativo



In [ ]:
# Gráfico de barras de la variable Danger
ggplot(sleep_limpio, aes(x = Danger)) +
  geom_bar(fill = "lightblue", color="pink") +
  labs(title = "Frecuencia de la variable Danger")


In [ ]:
# Gráfico de barras de la variable Danger ordenado descendentemente
ggplot(sleep_limpio, aes(x = fct_infreq(Danger))) +
  geom_bar(fill = "coral") +
  labs(title = "Variable Danger ordenada por frecuencia")


In [ ]:
# Para graficar un pie-chart
# ggplo2 no ofrece directamente un pie-chart, por lo que se deben hacer algunos cálculos
# Paso 1: Calcular la tabla de frecuencias
tabla_pie <- sleep_limpio %>%
  count(Danger) %>%
  mutate(
    porcentaje = n / sum(n),
    etiqueta = paste0(round(porcentaje * 100, 1), "%")
  )

# Paso 2: Crear el gráfico de pastel
ggplot(tabla_pie, aes(x = "", y = porcentaje, fill = Danger)) +
  geom_bar(stat = "identity", width = 1) +  # Dibujar barra circular con valores reales
  coord_polar("y", start = 0) +             # Transformar las barras en gráfico circular (pie chart)

  # Agregar etiquetas de porcentaje centradas en cada porción
  geom_text(aes(label = etiqueta),
            position = position_stack(vjust = 0.5),  # Centrar la etiqueta en la porción
            size = 5) +                              # Tamaño del texto de la etiqueta

  # Agregar títulos y leyenda
  labs(title = "Distribución de Danger", x = NULL, y = NULL, fill = "Danger") +

  theme_void() +  # Eliminar ejes y fondos para estilo limpio (propio de gráficos circulares)

  # Modificar el tamaño del título y leyenda
  theme(
    plot.title = element_text(size = 16, face = "bold"),  # Título grande y en negrita
    legend.title = element_text(size = 12),               # Título de la leyenda
    legend.text = element_text(size = 11)                 # Texto de las categorías en la leyenda
  )


In [ ]:
#Diagarama de barras de la bariable Exp ordenado por frecuencia descendente
ggplot(sleep_limpio, aes(x = fct_infreq(Exp))) +
  geom_bar(fill = "purple") +
  labs(title = "Variable Exp ordenada por frecuencia")

In [ ]:
#Diagrama de frecuencias variable Pred
ggplot(sleep_limpio, aes(x = fct_infreq(Pred))) +
  geom_bar(fill = "yellow") +
  labs(title = "Variable Pred ordenada por frecuencia")

In [ ]:
#Para poder agregar la etiqueta de los porcentajes:

# Paso 1: Calcular las frecuencias relativas
tabla_pred <- sleep_limpio %>%
  count(Pred) %>%
  mutate(
    porcentaje = n / sum(n),
    etiqueta = paste0(round(porcentaje * 100, 1), "%")
  )



# Paso 2: Gráfico con etiquetas
ggplot(tabla_pred, aes(x = Pred, y = porcentaje)) +
  geom_col(fill = "yellow") +
  geom_text(aes(label = etiqueta), vjust = -0.5) +
  scale_y_continuous(labels = scales::percent_format(accuracy = 1)) +
  labs(title = "Frecuencia relativa de la variable Pred",
       x = "Pred", y = "Porcentaje") +
  theme_minimal()

### Tablas cruzadas

In [ ]:
# Tabla cruzada entre dos variables categóricas
# Tabla cruzada con porcentajes por fila y renombrado
sleep_limpio %>%
  tabyl(Pred, Danger) %>%
  adorn_percentages("row")



In [ ]:
#Diagrama de barras con dos variables categóricas e inclusión de la frecuencia

# Paso 1: Calcular proporciones por grupo
tabla_prop <- sleep_limpio %>%
  count(Pred, Danger) %>%
  group_by(Pred) %>%
  mutate(
    prop = n / sum(n),
    etiqueta = paste0(round(prop * 100, 1), "%")
  )

# Paso 2: Gráfico de barras apiladas con etiquetas
ggplot(tabla_prop, aes(x = Pred, y = prop, fill = Danger)) +
  geom_col(position = "fill") +
  geom_text(aes(label = etiqueta),
            position = position_fill(vjust = 0.5), size = 3) +
  scale_y_continuous(labels = scales::percent_format(accuracy = 1)) +
  labs(title = "Relación entre Pred y Danger",
       x = "Tipo de depredador", y = "Proporción") +
  theme_minimal()


In [ ]:
# Análisis de asociación o relación entre dos variables categóricas
chisq.test(table(sleep_limpio$Pred, sleep_limpio$Danger))

El test de Chi-cuadrado asume que:

Al menos el 80% de las celdas deben tener frecuencias esperadas ≥ 5.

Ninguna celda debería tener una frecuencia esperada menor que 1.

Si esto no se cumple, la aproximación al valor-p usando la distribución Chi² puede no ser confiable, y por eso R nos advierte.



## Distribuciones de probabilidad


¿Qué es una distribución de probabilidad?

Una distribución de probabilidad describe cómo se distribuyen los valores posibles de una variable aleatoria. Es un modelo matemático que asigna una probabilidad a cada valor (discreto) o rango de valores (continuo).

Para variables discretas, usamos funciones de masa de probabilidad (como binomial o Poisson).
Para variables continuas, usamos funciones de densidad (como la normal o exponencial).






### Distribución normal:

Es una de las más comunes y útiles en estadística:

Tiene forma de campana simétrica.

Se define por dos parámetros:

* Media (μ)
* Desviación estándar (σ)

**Ejemplo:**

Comparar la distribución empírica de una variable cuantitativa con la teórica normal, usando ggplot2.

Usaremos la variable log_BodyWgt (peso logarítmico), ya que anteriormente mostramos que la transformación mejora la simetría y reduce outliers.

In [ ]:
# Crear gráfico con histograma y curva de densidad normal teórica
ggplot(sleep_limpio_BodyWgt, aes(x = log_BodyWgt)) +
  # Histograma con densidad (no frecuencia)
  geom_histogram(aes(y = ..density..), fill = "lightblue", bins = 30) +

  # Curva de densidad normal teórica ajustada a los datos
  stat_function(fun = dnorm,
                args = list(
                  mean = mean(sleep_limpio_BodyWgt$log_BodyWgt, na.rm = TRUE),
                  sd = sd(sleep_limpio_BodyWgt$log_BodyWgt, na.rm = TRUE)
                ),
                color = "red", size = 1) +

  # Etiquetas y estilo
  labs(title = "Comparación con distribución normal",
       x = "log(BodyWgt)", y = "Densidad") +
  theme_minimal()


--------------
| Función   | ¿Qué hace?                       | Ejemplo práctico                |
| --------- | -------------------------------- | ------------------------------- |
| `rnorm()` | Genera datos aleatorios          | Simulación de muestras          |
| `dnorm()` | Densidad (altura de la curva)    | Dibujar curva normal            |
| `pnorm()` | Probabilidad acumulada           | Calcular $P(X < x)$             |
| `qnorm()` | Cuantil inverso (valor de corte) | Buscar z tal que $P(X < z) = p$ |

-------------

**Ejercicio**:

Se sabe que el tiempo promedio que duerme un mamífero adulto es de 10 horas al día, con una desviación estándar de 2 horas.
Se asume que los tiempos de sueño siguen una distribución normal.


Grafica la distribución normal con μ=10 y σ=2.

Calcula las siguientes probabilidades:
* Que un mamífero duerma más de 12 horas al día.
* Que un mamífero duerma entre 8 y 12 horas.
* Que un mamífero duerma menos de 6 horas.

Visualiza las áreas bajo la curva correspondientes a cada probabilidad.

Interpreta los resultados.

In [ ]:
# Parámetros de la distribución
mu <- 10      # media
sigma <- 2    # desviación estándar

# Secuencia de valores x para graficar la curva
x_vals <- seq(0, 20, by = 0.1)

# Crear curva de densidad


ggplot(data.frame(x = x_vals), aes(x)) +
  stat_function(fun = dnorm, args = list(mean = mu, sd = sigma),
                color = "blue", size = 1) +
  labs(title = "Distribución Normal (μ = 10, σ = 2)",
       x = "Horas de sueño", y = "Densidad") +
  theme_minimal()

# Cálculo de las probabilidades:
# a) P(X > 12)
p_mayor_12 <- pnorm(12, mean = mu, sd = sigma, lower.tail = FALSE)

# b) P(8 < X < 12)
p_entre_8_12 <- pnorm(12, mean = mu, sd = sigma) - pnorm(8, mean = mu, sd = sigma)

# c) P(X < 6)
p_menor_6 <- pnorm(6, mean = mu, sd = sigma)

# Resultados
cat("P(X > 12):", round(p_mayor_12, 4), "\n")
cat("P(8 < X < 12):", round(p_entre_8_12, 4), "\n")
cat("P(X < 6):", round(p_menor_6, 4), "\n")


### Distribución Binomial

Modela el número de éxitos en un número fijo de ensayos independientes con dos posibles resultados: éxito o fracaso.

**Ejemplo típico:**

Lanzar una moneda 10 veces.
Probabilidad de obtener 6 caras.

Parámetros de esta distribución:
* n = número de ensayos
* p = probabilidad de éxito

In [ ]:
# Distribución binomial: 20 ensayos, probabilidad de éxito 0.3
binom_df <- data.frame(
  x = 0:20, #valores de 0-20
  prob = dbinom(0:20, size = 20, prob = 0.3)  # Probabilidad para cada valor
)

# Gráfico de barras de la distribución
ggplot(binom_df, aes(x = factor(x), y = prob)) +
  geom_bar(stat = "identity", fill = "skyblue") +
  labs(title = "Distribución Binomial (n=20, p=0.3)",
       x = "Número de éxitos", y = "Probabilidad") +
  theme_minimal()


-------------------
| Función                 | Qué hace                                              | Ejemplo                              |
| ----------------------- | ----------------------------------------------------- | ------------------------------------ |
| `rbinom(n, size, prob)` | Simula datos binomiales (n muestras de tamaño `size`) | `rbinom(10, size = 20, prob = 0.5)`  |
| `dbinom(x, size, prob)` | Probabilidad exacta $P(X = x)$                        | `dbinom(5, size = 10, prob = 0.5)`   |
| `pbinom(q, size, prob)` | Probabilidad acumulada $P(X \leq q)$                  | `pbinom(6, 10, 0.5)` → $P(X \leq 6)$ |
| `qbinom(p, size, prob)` | Cuantil: valor $x$ tal que $P(X \leq x) = p$          | `qbinom(0.95, 10, 0.5)`              |

-------------


**Ejercicio**

Un grupo de biólogos está evaluando la peligrosidad de ciertas especies animales. De acuerdo con registros previos, aproximadamente el 40% de los animales evaluados son considerados peligrosos. Se eligen 20 animales al azar.

Queremos modelar este proceso como un experimento binomial con: n=20 ensayos (animales seleccionados)
y p=0.40 la probabilidad de éxito (que un animal sea peligroso)

Para ello, simula la distribución binomial correspondiente al número de animales peligrosos esperados en una muestra de 20.

Visualiza la distribución usando un gráfico de barras.

Calcula la probabilidad de que:
* Exactamente 8 animales sean peligrosos.
* Menos de 5 animales sean peligrosos.
* Al menos 10 animales sean peligrosos.
Interpreta los resultados.

In [ ]:
# Parámetros
n <- 20      # número de ensayos
p <- 0.40    # probabilidad de éxito

# Crear distribución binomial
binom_df <- data.frame(
  x = 0:n,
  prob = dbinom(0:n, size = n, prob = p)
)

# Visualización
ggplot(binom_df, aes(x = x, y = prob)) +
  geom_col(fill = "skyblue", color = "black") +
  labs(title = "Distribución Binomial: n = 20, p = 0.40",
       x = "Número de animales peligrosos",
       y = "Probabilidad") +
  theme_minimal()

# Cálculos de probabilidades específicas
prob_8 <- dbinom(8, size = n, prob = p)
prob_menor5 <- pbinom(4, size = n, prob = p)
prob_al_menos10 <- 1 - pbinom(9, size = n, prob = p)

# Mostrar resultados
cat("P(X = 8):", round(prob_8, 4), "\n")
cat("P(X < 5):", round(prob_menor5, 4), "\n")
cat("P(X ≥ 10):", round(prob_al_menos10, 4), "\n")


### Distribución de Poisson

Modela el número de eventos que ocurren en un intervalo de tiempo o espacio cuando son raros y aleatorios.

**Ejemplo típico:**

* Número de llamadas por minuto a un centro de atención.
* Número de accidentes en una carretera por día.

Parámetro:

* λ = número medio de eventos en el intervalo.
* Si, X∼Poisson(λ), entonces la esperanza y varianza son iguales a λ.

In [ ]:
# Distribución de Poisson con media 5
poisson_df <- data.frame(
  x = 0:15, # valores entre 0 y 15
  prob = dpois(0:15, lambda = 5)
)

ggplot(poisson_df, aes(x = factor(x), y = prob)) +
  geom_bar(stat = "identity", fill = "lightgreen") +
  labs(title = "Distribución de Poisson (λ = 5)",
       x = "Número de eventos", y = "Probabilidad") +
  theme_minimal()


--------------------

| Función            | Qué hace                                               | Ejemplo                  |
| ------------------ | ------------------------------------------------------ | ------------------------ |
| `rpois(n, lambda)` | Simula conteos con media $\lambda$                     | `rpois(10, lambda = 4)`  |
| `dpois(x, lambda)` | Probabilidad exacta $P(X = x)$                         | `dpois(2, lambda = 3)`   |
| `ppois(q, lambda)` | Probabilidad acumulada $P(X \leq q)$                   | `ppois(4, lambda = 2.5)` |
| `qpois(p, lambda)` | Cuantil: valor mínimo $x$ tal que $P(X \leq x) \geq p$ | `qpois(0.9, lambda = 3)` |


--------------------


**Ejercicio**:

Un investigador estudia el número de despertares breves que tienen ciertos animales durante el sueño. Se ha observado que, en promedio, ocurren 3 despertares por noche.

Queremos modelar este proceso como una distribución de Poisson, con: λ=3 número medio de eventos por unidad de tiempo (noche)


Crea la distribución de Poisson con λ=3 para los valores posibles de 0 a 10 despertares.

Visualiza la distribución mediante un gráfico de barras.

Calcula las siguientes probabilidades:

* Que haya exactamente 2 despertares.
* Que haya 4 o más despertares.
* Que haya entre 1 y 3 despertares (inclusive).

Interpreta los resultados.

In [ ]:
# Parámetro de la distribución
lambda <- 3

# Valores posibles de X (número de despertares)
x_vals <- 0:10

# Crear tabla de probabilidades
poisson_df <- data.frame(
  x = x_vals,
  prob = dpois(x_vals, lambda = lambda)
)

ggplot(poisson_df, aes(x = factor(x), y = prob)) +
  geom_col(fill = "lightblue", color = "black") +
  labs(title = "Distribución de Poisson (λ = 3)",
       x = "Número de despertares por noche",
       y = "Probabilidad") +
  theme_minimal()

# Cálculo de las probabilidades:

# a) P(X = 2)
p_2 <- dpois(2, lambda = lambda)

# b) P(X ≥ 4)
p_mayor_igual_4 <- ppois(3, lambda = lambda, lower.tail = FALSE)

# c) P(1 ≤ X ≤ 3)
p_entre_1_3 <- ppois(3, lambda = lambda) - ppois(0, lambda = lambda)

# Mostrar resultados
cat("P(X = 2):", round(p_2, 4), "\n")
cat("P(X ≥ 4):", round(p_mayor_igual_4, 4), "\n")
cat("P(1 ≤ X ≤ 3):", round(p_entre_1_3, 4), "\n")


### Distribución exponencial:

La distribución exponencial es usada para modelar el tiempo entre eventos en un proceso que ocurre de manera aleatoria y continua a una tasa constante.

Tiene un solo parámetro: λ (lambda) = tasa de ocurrencia de eventos por unidad de tiempo

- **Media**: 1/λ

- **Varianza**: 1/λ^2


In [ ]:
set.seed(123)  # reproducibilidad

# Simular 200 tiempos entre eventos con tasa lambda = 0.5
# (es decir, en promedio un evento cada 2 unidades de tiempo)
exp_data <- rexp(n = 200, rate = 0.5)

# Crear dataframe
exp_df <- data.frame(tiempo = exp_data)

ggplot(exp_df, aes(x = tiempo)) +
  geom_histogram(aes(y = ..density..), bins = 30, fill = "lightgreen", color = "black") +
  stat_function(fun = dexp, args = list(rate = 0.5), color = "red", size = 1) +
  labs(
    title = "Simulación de tiempos ~ Distribución Exponencial (λ = 0.5)",
    x = "Tiempo entre eventos", y = "Densidad"
  ) +
  theme_minimal()


--------------------
| Función         | Qué hace                                     | Ejemplo                 |
| --------------- | -------------------------------------------- | ----------------------- |
| `rexp(n, rate)` | Simula datos de tiempos de espera            | `rexp(10, rate = 0.2)`  |
| `dexp(x, rate)` | Densidad: valor de $f(x)$                    | `dexp(3, rate = 0.5)`   |
| `pexp(q, rate)` | Probabilidad acumulada $P(X \leq q)$         | `pexp(4, rate = 0.2)`   |
| `qexp(p, rate)` | Cuantil: valor $x$ tal que $P(X \leq x) = p$ | `qexp(0.9, rate = 0.2)` |

--------------------

Ejercicio:

Simula una variable que represente los tiempos entre visitas a una estación de observación de animales. Supón que los eventos ocurren a una tasa constante de 0.25 visitas por hora.

Simula 300 observaciones usando una distribución exponencial con rate = 0.25.

Grafica el histograma junto con la curva teórica de densidad.

Calcula la media y la varianza observadas.

Compara tus resultados con los valores teóricos.

Ajusta un nuevo valor de lambda y repite los pasos anteriores. ¿Cómo cambia la distribución?

In [ ]:
# Paso 1: Simular
set.seed(456)
tiempos <- rexp(n = 300, rate = 0.25)
df_tiempos <- data.frame(tiempo = tiempos)

# Paso 2: Visualizar
ggplot(df_tiempos, aes(x = tiempo)) +
  geom_histogram(aes(y = ..density..), bins = 30, fill = "lightblue", color = "black") +
  stat_function(fun = dexp, args = list(rate = 0.25), color = "red") +
  labs(title = "Tiempos entre visitas (λ = 0.25)",
       x = "Tiempo (horas)", y = "Densidad") +
  theme_minimal()

# Paso 3: Estadísticos
mean(df_tiempos$tiempo)
var(df_tiempos$tiempo)


In [ ]:
# valor distinto de la tasa:

# Paso 1: Simular
set.seed(456)
tiempos <- rexp(n = 300, rate = 0.75)
df_tiempos <- data.frame(tiempo = tiempos)

# Paso 2: Visualizar
ggplot(df_tiempos, aes(x = tiempo)) +
  geom_histogram(aes(y = ..density..), bins = 30, fill = "lightblue", color = "black") +
  stat_function(fun = dexp, args = list(rate = 0.25), color = "red") +
  labs(title = "Tiempos entre visitas (λ = 0.25)",
       x = "Tiempo (horas)", y = "Densidad") +
  theme_minimal()

# Paso 3: Estadísticos
mean(df_tiempos$tiempo)
var(df_tiempos$tiempo)


# Evaluación final

